In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from utility import *
from botorch.acquisition.analytic import ExpectedImprovement,ProbabilityOfImprovement,UpperConfidenceBound

In [2]:
X_train = pd.read_csv('./input/X_train.csv')
y_train = pd.read_csv('./input/y_train.csv').iloc[:,0]
parameter = pd.read_csv('./input/parameters.csv')
score = -pd.read_csv('./input/scores.csv').iloc[:,1:]

In [3]:
log_rmse = lambda yhat,y: np.sqrt(np.mean((np.log(1+y) - np.log(1+yhat))**2))
def xgboost2perf(learning_rate,max_depth,min_child_weight,gamma,alpha,reg_lambda,\
                 subsample,colsample_bytree,objective,tree_method):

    xgboost = XGBRegressor(learning_rate=learning_rate,\
                           n_estimators=600,\
                           nthread=-1,\
                           
                           max_depth=max_depth,\
                           min_child_weight=min_child_weight,\
                           gamma=gamma,\
                           alpha=alpha,\
                           reg_lambda=reg_lambda,\
                           subsample=subsample,\
                           colsample_bytree=colsample_bytree,\
                           objective=objective,\
                           tree_method=tree_method
                           )

    score = 0
    n_fold = 6
    kf = KFold(n_fold,shuffle=True,random_state=7)
    for train_index, test_index in kf.split(X_train):
        xgboost.fit(X_train.iloc[train_index],y_train.iloc[train_index]);
        yhat = np.maximum(xgboost.predict(X_train.iloc[test_index]),0)
        score += log_rmse(yhat,y_train.iloc[test_index])
    score /= n_fold
    return -score # min -> max

In [4]:
# 'reg:squaredlogerror' and 'reg:pseudohubererror' does not work
cat_feat = {'objective':{'reg:squarederror':0,'reg:absoluteerror':1,'reg:gamma':2,'count:poisson':3},\
            'tree_method':{'exact':0,'hist':1}}
bounds = [[0.1,0.6],[2,16],[1.0,100.0],\
          [0.0,100.0],[0.0,100.0],[0.0,100.0],\
          [0.25,1.0],[0.25,1.0]]
IsMax = False
integer = [1,]

T = Transform(cat_feat,integer,bounds)
bo_parameters = {'acq_fun': ExpectedImprovement,\
                'acq_kwargs': {},\
                 #'fit': fit_gpytorch_mll,\
                'BaseKernel': MaternKernel,\
                'eps': 0.8,\
                'STE':True,\
                'beta':10.0,\
                'b':1,\
                'q': 1,\
                'num_restarts': 12,\
                'raw_samples': 256,\
                'Bo_iter': 100,\
                'verbose': 10
                }

In [ ]:
#%debug
import warnings
warnings.filterwarnings('ignore')
x_best,y_best,x,y,model = BO(xgboost2perf,parameter,score,T,**bo_parameters)

best val is -0.15011900523637314 at iter 10
best val is -0.14489240629762154 at iter 20
best val is -0.1354507588669662 at iter 30
best val is -0.12449252396658489 at iter 40
best val is -0.12449252396658489 at iter 50
best val is -0.12449252396658489 at iter 60
best val is -0.12449252396658489 at iter 70


In [7]:
x_best,y_best,x,y,model = BO(xgboost2perf,x,y,T,**bo_parameters)

best val is -0.12451533314999341 at iter 10
best val is -0.12451533314999341 at iter 20
best val is -0.12451533314999341 at iter 30
best val is -0.12327934133156083 at iter 40
best val is -0.12291377232673158 at iter 50
best val is -0.1222194477034604 at iter 60
best val is -0.1222194477034604 at iter 70
best val is -0.1222194477034604 at iter 80
best val is -0.1222194477034604 at iter 90
best val is -0.1222194477034604 at iter 100


In [9]:
pd.concat([x,y],1).sort_values(by=['scores'],ascending=False)

,learning_rate,max_depth,min_child_weight,gamma,alpha,reg_lambda,subsample,colsample_bytree,objective,tree_method,scores
0,0.100000,5,51.834650,71.345106,9.684210,80.049663,0.926508,0.499708,count:poisson,hist,-0.122219
0,0.100000,4,32.404418,90.128073,59.003655,22.550415,0.895481,0.513877,count:poisson,hist,-0.122276
0,0.100000,4,83.933899,95.610325,13.814104,77.801405,0.796111,0.436662,count:poisson,hist,-0.122914
0,0.100000,4,32.992335,100.000000,6.137757,21.815003,0.884555,0.576040,count:poisson,hist,-0.123257
0,0.100000,4,30.827384,65.492642,91.035223,56.297423,0.838361,0.568791,count:poisson,hist,-0.123279
...,...,...,...,...,...,...,...,...,...,...,...
6,0.326054,3,11.919477,28.552929,76.890287,54.607057,0.502535,0.795976,reg:absoluteerror,hist,-0.284340
9,0.427221,3,80.484861,47.357331,82.625859,91.327121,0.375483,0.492813,reg:absoluteerror,exact,-0.400163
1,0.280107,2,79.122748,94.706736,85.902933,7.053940,0.282078,0.522069,reg:absoluteerror,hist,-0.401253
2,0.403225,3,38.475863,72.333209,41.717891,17.955389,0.535492,0.305701,reg:gamma,exact,-0.402244


In [22]:
x_best,y_best,x,y,model = BO(xgboost2perf,x,y,T,**bo_parameters)

best val is -0.1270606684379154 at iter 10
best val is -0.1270606684379154 at iter 20
best val is -0.1270606684379154 at iter 30
best val is -0.1270606684379154 at iter 40


KeyboardInterrupt: 

In [36]:
pd.concat([x,y],1).sort_values(by=['scores'],ascending=False).iloc[0:1,:-1].to_csv('best_para.csv',index=False)

In [18]:
# x_tor,y_tor = T.forward(x,y)

# model = initialize_model(x_tor,y_tor, MaternKernel)
# h = model.register_forward_pre_hook(hook_factory(T))

# d0 = 8;d1 = 6;
# bounds = torch.tensor([[0.0] * d0 + [-10] * d1, [1.0] * d0 + [10] * d1], device=device, dtype=dtype)    
# bounds2 = torch.tensor([[0.0] * (d0+d1), [1.0] * (d0+d1)], device=device, dtype=dtype)    

# from botorch.acquisition.analytic import AnalyticAcquisitionFunction
# class Mean_std(AnalyticAcquisitionFunction):
#     def __init__(self,model,beta) -> None:
#         super().__init__(model=model)
#         self.beta = beta

#     def forward(self, X):
#         mean, std = self._mean_and_sigma(X, compute_sigma=True)
#         return mean - self.beta * std

# mean_fun = Mean_std(model,0)
# #mean_fun = ExpectedImprovement(model,-y.min().values[0])

# #%debug
# x_best,y_val = optimize_acqf(mean_fun,bounds2,q=1,num_restarts=256,raw_samples=1024)
# print(y_val)
# print(xgboost2perf(*T.backward(x_best[0])))

# x_best,y_val = optimize_acqf(mean_fun,bounds,q=1,num_restarts=256,raw_samples=1024)
# print(y_val)
# print(xgboost2perf(*T.backward(x_best[0])))

In [10]:
bo_parameters = {'acq_fun': UpperConfidenceBound,\
                'acq_kwargs': {'beta':6},\
                 #'fit': fit_gpytorch_mll,\
                'BaseKernel': RBFKernel,\
                'eps': 0.6,\
                'STE':True,\
                'beta':2.0,\
                'q': 1,\
                'num_restarts': 12,\
                'raw_samples': 256,\
                'Bo_iter': 100,\
                'verbose': 10
                }

In [ ]:
x_best,y_best,x,y,model = BO(xgboost2perf,x,y,T,**bo_parameters)

best val is -0.1222194477034604 at iter 10
best val is -0.1222194477034604 at iter 20
best val is -0.1222194477034604 at iter 30


In [ ]:
pd.concat([x,y],1).sort_values(by=['scores'],ascending=False)

In [12]:
x.to_csv('bo_x.csv'); y.to_csv('bo_y.csv')